# Lab1 Julia language
## Preliminary explanations

Measuring the working time is not as trivial as it seems at first glance. In the Julia language, it is further complicated by the fact that the compilation stage is hidden from the user. It occurs at the first start of the program and can take a long time, so to properly measure the running time, you should follow the following rules.

* Separate the code whose performance you want to measure into a separate function
* Always measure the operation of this function, not the entire program
* Before measuring, call the function at least once so that the compiler compiles its code in advance.

sequential programming/procedural programming: 

## Task 1
Any program that uses parallel computing makes sense only if it runs faster than a program that does the same thing, but sequentially. So it is extremely important to measure the execution time. Before measuring the running time of parallel programs, it is necessary to learn how to correctly measure the running time of sequential programs. In Julia, the task is facilitated by the availability of ready-made functions and macros for these purposes.
* Write a simple routine that waits for some time, for example 1 second. To wait, use the built-in ```sleep``` function. Is it possible to pass fractional values to it? Read the help of the ```time_ns``` function and replace the ```sleep``` function with it.
* Measure the operation time of the function 10 times by printing out the results. Use ```@time```. Is the execution time different or is it the same all the time?
* Take time measurements for ```1E6``` launches. You won't be able to wait one second, so replace ```sleep``` with ```time_ns```. Measure the running time of the program, print it out as a dot cloud (scatter) and a histogram.

To measure the time, you can use ```BenchmarkTools```, which can build histograms of the running time of functions, but you need to figure out how it works.

#### Answer: 

* According to the official documentation of ```sleep(seconds)```, the functions accept minimum sleep time 1 milisecond or the fractional input of 0.001.

In [1]:
function normal_sleep(seconds)
    sleep(seconds) # second должно быть не меньше 1 миллисекунды или 0,001 секунды
end

normal_sleep (generic function with 1 method)

In [2]:
normal_sleep(1)

In [3]:
normal_sleep(10)
# sleep for 10 second

In [4]:
normal_sleep(0.001)
# sleep for 1 milisecond

In [5]:
normal_sleep(0.00001)
# sleep for less than 1 milisecond

In [6]:
# we can not see on the the IDE the execution time oeach routine call, but will see with @time that normal_sleep(0.001) and normal_sleep(0.000001) will be the same. 
# first call, and first use of @time macro, so take longer, will use it again.
@time normal_sleep(1)

  1.016420 seconds (38 allocations: 1.109 KiB)


In [7]:
@time normal_sleep(0.001)

  0.011405 seconds (7 allocations: 192 bytes)


In [8]:
@time normal_sleep(0.0001)
@time normal_sleep(0.00001)
@time normal_sleep(0.000001)
@time normal_sleep(0.0000001)
@time normal_sleep(0.00000001)

  0.016902 seconds (7 allocations: 192 bytes)
  0.006511 seconds (13 allocations: 432 bytes)


  0.002396 seconds (59 allocations: 3.867 KiB)
  0.005945 seconds (13 allocations: 544 bytes)
  0.018705 seconds (6 allocations: 240 bytes)


In [9]:
# using @timed for smaller time measurement and better display
    for i in 1:9
        sleeping_time = 1 / (10^(i))
        println("sleeping_time= $sleeping_time seconds => execution time = ", (@timed normal_sleep(sleeping_time)).time)
    end

sleeping_time= 0.1 seconds => execution time = 0.1069833
sleeping_time= 0.01 seconds => execution time = 0.0166828
sleeping_time= 0.001 seconds => execution time = 0.0119512
sleeping_time= 0.0001 seconds => execution time = 0.0150089
sleeping_time= 1.0e-5 seconds => execution time = 0.0060794


sleeping_time= 1.0e-6 seconds => execution time = 0.0147009
sleeping_time= 1.0e-7 seconds => execution time = 0.0166858
sleeping_time= 1.0e-8 seconds => execution time = 0.0030295
sleeping_time= 1.0e-9 seconds => execution time = 0.0152441


```time_ns()``` return current time in nanosecond ($10^{-9}$ seconds) which give more precise timing. However this method does not block a task directly that ```sleep()``` does. To do something similar, or to replace ```sleep()``` with the use of ```time_ns()``` and accept fractional input as small as ($10^{-9}$ seconds). Let's implement a function to block_task by using using time_ns() for smaller blocking time instead of ```sleep()```.

In [10]:
function time_ns_sleep(second)
    starter = time_ns()
    while (time_ns() - starter) < (second*1e9)
        # do nothing, just wait!
        # elapsed time
    end
end

time_ns_sleep (generic function with 1 method)

In [11]:
@time time_ns_sleep(1)

  1.000003 seconds


In [ ]:
@time time_ns_sleep(0.001)
@time time_ns_sleep(0.0001)
@time time_ns_sleep(0.00001)
@time time_ns_sleep(0.000001)
@time time_ns_sleep(0.0000001)
@time time_ns_sleep(0.00000001)
@time time_ns_sleep(0.000000001)

  0.001017 seconds
  0.000101 seconds
  0.000011 seconds


  0.000003 seconds
  0.000001 seconds
  0.000001 seconds
  0.000001 seconds


In [13]:
# using @timed for smaller time measurement and better display
for i in 1:9
    sleeping_time = 1 / (10^(i))
    println("sleeping_time= $sleeping_time seconds => execution time = ", (@timed time_ns_sleep(sleeping_time)).time)
end

sleeping_time= 0.1 seconds => execution time = 0.1000002
sleeping_time= 0.01 seconds => execution time = 0.0100001
sleeping_time= 0.001 seconds => execution time = 0.0010001
sleeping_time= 0.0001 seconds => execution time = 0.0001001
sleeping_time= 1.0e-5 seconds => execution time = 1.0e-5
sleeping_time= 1.0e-6 seconds => execution time = 1.1e-6
sleeping_time= 1.0e-7 seconds => execution time = 1.0e-7
sleeping_time= 1.0e-8 seconds => execution time = 1.0e-7
sleeping_time= 1.0e-9 seconds => execution time = 2.0e-7


Test @time of each function 10 time

In [14]:
for i in 1:10
    println("Run $i")
    @time time_ns_sleep(1e-4)
end

Run 1
  0.000101 seconds
Run 2
  0.000100 seconds
Run 3
  0.000100 seconds
Run 4
  0.000100 seconds
Run 5
  0.000101 seconds
Run 6
  0.000100 seconds
Run 7
  0.000101 seconds
Run 8
  0.000100 seconds
Run 9
  0.000101 seconds
Run 10
  0.000100 seconds


Since sleep() accept minimum value of 1 milisecond only, the execution time of ```normal_sleep()``` remain in this order of magnitude and it change. 

For this test, the sleeping time used in ```time_ns()``` related function is 1 nano second. Thus, it really has little impact on the execution time of the overal routine which are in the order of 0.01 milisecond. 

Let's @time the routine with ```time_ns()``` for 1E6 times. However, since the built-in method ```@time``` return the execution time, gabage collection , memory allocation all in a formtted string, let's us use ```@timed``` instead which give the same result but in form of tupple. 

In [15]:
measureds = []
measurements = []
function get_time()
    # test a routine that sleep for 1 microsecond.
    # 1e-6 = 0.000001 second
    return (@timed time_ns_sleep(1e-6)).time 
end

N = Integer(1e6)

for i in 1:N
    push!(measureds, get_time())
    push!(measurements, i)
end

In [27]:
using Plots;
gr(size=(1000, 600))
scatter(measurements, measureds, label="measured execution time", ms=0.4, ma=0.7, color="blue")
xlabel!("Order of launches")
ylabel!("Measured execution time")
title!("scatter plot of execution time over $N launches")
savefig("scatt.png")

"d:\\work\\study\\2023-2024\\Параллельное программирование\\lab01\\scatt.png"

code tends to stuck here if not savefigure to file. 

In [24]:
histogram(measureds, label="Measured execution time", color=:blue)
xlabel!("Execution time in seconds")
ylabel!("Counts")
title!("histogram of execution time over $N launches")
savefig("histogram.png")

"d:\\work\\study\\2023-2024\\Параллельное программирование\\lab01\\histogram.png"

In [19]:
using BenchmarkTools;
using Plots;
benchmark = @benchmarkable time_ns_sleep(1e-6) evals= 1e6
run(benchmark)

BenchmarkTools.Trial: 5 samples with 1000000 evaluations.
 Range (min … max):  1.009 μs … 1.029 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.011 μs             ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.015 μs ± 8.240 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █    ██ █                                               █  
  █▁▁▁▁██▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.01 μs        Histogram: frequency by time       1.03 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [ ]:
# data cleansing
using Statistics;
mean_ = mean(measureds)
for i in 1:N
    if abs(measureds[i] - mean_) > 0.5e-5
        measureds[i] = mean_ + 0.5e-5
    end
end

## Task2
There is no OpenMP implementation in Julia, but the concepts underlying multithreading are common in both Julia and OpenMP.

* Create a program that generates threads. Print out the number of threads created. With what function is this done?
* Julia allows you to set the number of threads at the start time. There are several ways to do this: using an environment variable and using a command line parameter. Create examples illustrating both ways. Run the program several times, each time changing the number of threads and checking that the threads are running. The ```sleep()``` function may be useful.
* How many threads does your processor efficiently support? How can I find out this information? How can Julia automatically create the number of threads that the processor supports optimally?
* Create a multithreaded program with four threads that accepts an array of integers as input. You need to manually distribute the work between the threads. The first thread should sum 1, 5, 9, etc. numbers; the second thread — 2, 6, 10, etc.; the third — 3, 7, 11; the fourth — 4, 8, 12, etc. The summation results are printed out indicating which thread received which result.
* Write automatic tests for this program that check its performance for different sequences of numbers.

In [20]:
using Base.Threads

function create_threads()
    println("Number of threads created: ", Threads.nthreads())
end

# Call the function to create threads and print the count
create_threads()


Number of threads created: 1


To work with multithreading in Julia and achieve the tasks you've described, you can use the Threads module. Here's how you can address each of your points:

Create a program that generates threads:
To create threads in Julia and print the number of threads created, you can use the Threads.nthreads() function. Here's an example:

julia
Copy code
using Base.Threads

function create_threads()
    println("Number of threads created: ", Threads.nthreads())
    # Create and start threads here
end

create_threads()
Set the number of threads at the start time:
You can set the number of threads in Julia using both environment variables and command line parameters. Here's how to do it for both cases:

Using environment variables (e.g., JULIA_NUM_THREADS):

julia
Copy code
# Set the number of threads using an environment variable
ENV["JULIA_NUM_THREADS"] = "4"  # Set the desired number of threads
println("Number of threads (from environment variable): ", Threads.nthreads())
Using command line parameters:
You can pass the desired number of threads as a command line argument when running your Julia script. For example:

julia
Copy code
# Check for command line argument
if length(ARGS) > 0
    n_threads = parse(Int, ARGS[1])
    Threads.nthreads() != n_threads && Threads.nthreads(n_threads)
end
println("Number of threads (from command line parameter): ", Threads.nthreads())
You can run the program with different values of JULIA_NUM_THREADS or command line arguments to change the number of threads.

Determine the number of threads your processor supports efficiently:
To find out the optimal number of threads your processor can efficiently support, you can use the Sys.CPU_THREADS variable. You can set the number of threads based on this value like this:

julia
Copy code
desired_threads = Sys.CPU_THREADS
Threads.nthreads() != desired_threads && Threads.nthreads(desired_threads)
Julia can automatically set the number of threads to the optimal value by using the Threads.nthreads() function without specifying a number. This will generally use the number of CPU cores available on your system.

Create a multithreaded program with four threads:
Here's an example of a multithreaded program in Julia that distributes work between four threads as described:

julia
Copy code
using Base.Threads

function sum_thread(start::Int, step::Int, n::Int)
    my_sum = 0
    for i in start:step:n
        my_sum += i
    end
    return my_sum
end

function main()
    n = 100  # Adjust this as needed
    results = Vector{Int}(undef, 4)

    threads = [Thread() for _ in 1:4]

    for (i, thread) in enumerate(threads)
        t_start = i
        t_step = 4
        t = Thread(() -> results[i] = sum_thread(t_start, t_step, n))
        push!(thread, t)
    end

    for thread in threads
        start(thread)
    end

    for thread in threads
        wait(thread)
    end

    for (i, result) in enumerate(results)
        println("Thread $i: $result")
    end
end

main()
Write automatic tests:
You can use Julia's Test module to write tests for your multithreaded program. Here's an example of how you can test the program's performance:

julia
Copy code
using Test

@testset "Multithreaded Sum" begin
    function test_multithreaded_sum(n::Int)
        results = Vector{Int}(undef, 4)
        expected_results = [2500, 2500, 2500, 2500]

        threads = [Thread() for _ in 1:4]

        for (i, thread) in enumerate(threads)
            t_start = i
            t_step = 4
            t = Thread(() -> results[i] = sum_thread(t_start, t_step, n))
            push!(thread, t)
        end

        for thread in threads
            start(thread)
        end

        for thread in threads
            wait(thread)
        end

        @test results == expected_results
    end

    # Test with different values of 'n'
    test_multithreaded_sum(100)
    test_multithreaded_sum(1000)
    test_multithreaded_sum(10000)
end
This code defines a test function test_multithreaded_sum that checks if the results match the expected values for different input sizes. The @test macro is used to check the conditions. Adjust the test cases and expected results as needed.

### Analysis



## Task3
All built-in mathematical functions of the Julia language support vector actions with arrays, that is, if an array is passed to them as an argument, not a scalar value, then the function will be calculated from each element of the array. To do this, you need to specify a dot after the function name. For example, if we want to calculate the sine from each element of the array ```A```, we must call the function like this: ```sin.(A)```.
Potentially, such functions can use the processor's SIMD registers, which should give a performance gain compared to calculations in a loop.
As a third task, come up with some of your own function that manipulates a numeric array and calculates some values. Compose it from elementary mathematical functions and calculate its value from the array in two ways.

* The first method is to calculate the values of functions from array elements in a loop, passing each element of the array to the function separately.
* The second method is to pass the entire array as an argument.

The second method should theoretically give a performance gain. Measure the execution time using ```@time```.

Read in the official documentation about the macros ```@inbounds, @fastmath and @simd``` and apply them in your loop. Be sure to check the correctness of calculations when using ```@fastmath``` (how to do this?)


## Task4
Julia has a large collection of built-in functions that allow you to write code in a functional style. The code written in this style lends itself very well to parallelization, since the functions used are clean and do not have to modify their arguments, which prevents a data race.

* Learn the ```reduce``` function. Tell us in your own words how it works.
* Create a small array of integers, such that you can check the correctness of the calculations.
Use ```reduce``` to do the following with it.
    - Find all positive numbers, negative numbers, even, odd, divisible without remainder by 7.
    - Then, with the sequences resulting from such filtering, perform the following operations: sum up, find the maximum, minimum, average, sample variance.
* Try to immediately create an array with the conditions listed above, that is, for example, one that consists of integers divisible by 7 without remainder.
* Combine all the conditions together, that is, for example, find the sum of all the elements that are divisible by 7 without remainder, while positive, no more than some number.
* The task can be completed in many ways. Laboratory protection will be easier if anonymous functions, the  operator  |> and functions from the module ```Base.Iterators``` are used. 

In addition, each task must be implemented in one line. That is, for example, creating an array, finding all positive numbers and summing them should take one line of code. Try not to use semicolons.

In [5]:
using Statistics  # Import the Statistics module for mean function

arr = [1, 2, -3, 4, 5, -6, 7, 14, 21]

# Find positive numbers, negative numbers, even, odd, and divisible by 7
positives = arr |> x -> filter(y -> y > 0, x)
negatives = arr |> x -> filter(y -> y < 0, x)
even_numbers = arr |> x -> filter(y -> y % 2 == 0, x)
odd_numbers = arr |> x -> filter(y -> y % 2 != 0, x)
divisible_by_7 = arr |> x -> filter(y -> y % 7 == 0, x)

# Perform operations on filtered sequences
sum_positives = positives |> x -> reduce(+, x)
max_negatives = negatives |> x -> reduce(max, x)
min_even = even_numbers |> x -> reduce(min, x)
average_odd = odd_numbers |> x -> mean(x)  # Use mean function from Statistics module
variance_div7 = divisible_by_7 |> x -> sum((i - mean(x))^2 for i in x) / (length(x) - 1)

# Print the results
println("Положительные числа: $positives")
println("Отрицательные числа: $negatives")
println("Четные числа: $even_numbers")
println("Нечетные числа: $odd_numbers")
println("Сумма положительных чисел: $sum_positives")
println("Максимальное количество отрицательных чисел: $max_negatives")
println("Минимум четных чисел: $min_even")
println("Среднее значение нечетных чисел: $average_odd")
println("Sample variance of numbers divisible by 7: $variance_div7")



Положительные числа: [1, 2, 4, 5, 7, 14, 21]
Отрицательные числа: [-3, -6]
Четные числа: [2, 4, -6, 14]
Нечетные числа: [1, -3, 5, 7, 21]
Сумма положительных чисел: 54
Максимальное количество отрицательных чисел: -3
Минимум четных чисел: -6
Среднее значение нечетных чисел: 6.2
Sample variance of numbers divisible by 7: 49.0


In [22]:
divisible_by_7_directly = [x for x in arr if x % 7 == 0]


3-element Vector{Int64}:
  7
 14
 21

In [23]:
sum_divisible_by_7_positive_limit = arr |> x -> filter(y -> y > 0 && y % 7 == 0, x) |> x -> sum(x)


42